In [9]:
import time
start = time.time()

import argparse
import cv2
import itertools
import os
import numpy as np
np.set_printoptions(precision=2)
import openface
import dlib

In [10]:
# From https://github.com/cmusatyalab/openface/blob/master/demos/compare.py

imgs = [] # input images
dlibFacePredictor = "/home/fien/openface/models/dlib/shape_predictor_68_face_landmarks.dat"
networkModel = "/home/fien/openface/models/openface/nn4.small2.v1.t7"
imgDim = 96
verbose = True

if verbose:
    print("Argument parsing and loading libraries took {} seconds.".format(
        time.time() - start))

start = time.time()
align = openface.AlignDlib(dlibFacePredictor)
net = openface.TorchNeuralNet(networkModel, imgDim)
   
if verbose:
    print("Loading the dlib and OpenFace models took {} seconds.".format(
        time.time() - start))

Argument parsing and loading libraries took 0.241365909576 seconds.
Loading the dlib and OpenFace models took 3.3763730526 seconds.


In [13]:
def getRep(imgPath):
    if verbose:
        print("Processing {}.".format(imgPath))
    bgrImg = cv2.imread(imgPath)
    if bgrImg is None:
        raise Exception("Unable to load image: {}".format(imgPath))
    rgbImg = cv2.cvtColor(bgrImg, cv2.COLOR_BGR2RGB)

    if verbose:
        print("  + Original size: {}".format(rgbImg.shape))

    start = time.time()
    bb = align.getLargestFaceBoundingBox(rgbImg)
    if bb is None:
        raise Exception("Unable to find a face: {}".format(imgPath))
    if verbose:
        print("  + Face detection took {} seconds.".format(time.time() - start))

    start = time.time()
    alignedFace = align.align(imgDim, rgbImg, bb,
                              landmarkIndices=openface.AlignDlib.OUTER_EYES_AND_NOSE)
    if alignedFace is None:
        raise Exception("Unable to align image: {}".format(imgPath))
    if verbose:
        print("  + Face alignment took {} seconds.".format(time.time() - start))

    start = time.time()
    rep = net.forward(alignedFace) 
    
    if verbose:
        print("  + OpenFace forward pass took {} seconds.".format(time.time() - start))
        print("Representation:")
        print(rep)
        print(" Length of the representation " + str(len(rep)) + " and type " + str(type(rep)))
        print("-----\n")
    return rep

In [14]:
img1 = "faces/face1.jpg"
img2 = "faces/face2.jpg"

d = getRep(img1) - getRep(img2)
print("Comparing {} with {}.".format(img1, img2))
print("  + Squared l2 distance between representations: {:0.3f}".format(np.dot(d, d)))

Processing faces/face1.jpg.
  + Original size: (512, 512, 3)
  + Face detection took 0.274261951447 seconds.
  + Face alignment took 0.0157899856567 seconds.
  + OpenFace forward pass took 0.119620084763 seconds.
Representation:
[ 0.07 -0.11  0.04  0.   -0.02  0.03 -0.    0.06 -0.02 -0.12  0.1   0.1
  0.07 -0.08 -0.07  0.08 -0.19 -0.13 -0.19  0.17  0.07 -0.03  0.08 -0.05
 -0.12  0.06  0.03 -0.04 -0.07 -0.06 -0.1   0.08  0.02  0.18 -0.16  0.02
 -0.04  0.09 -0.06  0.07  0.05 -0.16 -0.02 -0.04 -0.04 -0.08  0.02 -0.03
 -0.07  0.2  -0.02 -0.06 -0.16  0.16  0.1  -0.08 -0.03 -0.04 -0.11  0.1
 -0.04 -0.04  0.06  0.03  0.02 -0.01 -0.05  0.03 -0.01  0.03 -0.09 -0.03
 -0.    0.15  0.11 -0.03  0.11 -0.04 -0.05  0.1   0.07 -0.19  0.03 -0.02
 -0.14  0.13  0.09  0.03 -0.02  0.08  0.11 -0.15 -0.1   0.09 -0.12  0.07
 -0.08 -0.03  0.08  0.02 -0.01 -0.06 -0.08  0.07  0.15  0.04 -0.06  0.2
 -0.04 -0.15 -0.12  0.13 -0.02  0.01  0.06  0.03  0.1   0.08  0.06  0.16
  0.02  0.03  0.08  0.08  0.05  0.01  0.1   